In [1]:
import io
import sys
import pdfminer
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings 
warnings.filterwarnings('ignore')
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 6.0)


In [3]:
import spacy
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.graph_objs import FigureWidget
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()
from collections import Counter
import scattertext as st
import spacy
from pprint import pprint

Loading BokehJS ...

In [4]:
from wordcloud import WordCloud, STOPWORDS 


In [5]:
def convert(filename):
    fp = open(filename, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()

    return data

In [6]:
#data=convert('t.pdf')
import os

In [8]:
#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
def convertMultiple(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        fileName=pdf.split(".")[0]
        if fileExtension == "pdf":
            pdfFilename = pdfDir + pdf 
            text = convert(pdfFilename) #get string of text content of pdf
            textFilename = txtDir + fileName  + ".txt"
            textFile = open(textFilename, "w",encoding='utf8') #make text file
            textFile.write(text) #write text to text file
            textFile.close()

pdfDir = "C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/dataf/"
txtDir = "C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/"
resDir = "C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/"

convertMultiple(pdfDir, txtDir)

In [9]:
def remove_non_ascii(text):
    return ''.join(i for i in text if ord(i)<128)

In [10]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [14]:
textFilename

'C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/CN102666017B.txt'

In [20]:
df = pd.DataFrame()
df_c = pd.DataFrame()
out_data=pd.DataFrame()
out_data_clean=pd.DataFrame()

for file in os.listdir(txtDir):
    textFilename = txtDir + file
    print(textFilename)
    fileExtension = file.split(".")[-1]
    if fileExtension == "txt":
        data = pd.read_csv(textFilename,sep="/n", header = None)
        #data = pd.read_csv(textFilename,delim_whitespace=True, header = None)
        df=df.append(data)
        df_c = pd.concat([df_c, data], ignore_index=True,axis=1)
        
        data.dropna(inplace=True)
        data.columns=['text']
        #data[~data.text.str.contains("cid")]
        
        data_replace = data['text'].str.replace('\d+', '')
        data_replace = data_replace.apply(remove_non_ascii)
        #data_replace[~data_replace.str.contains("cid")]
        
        common_words = get_top_n_bigram(data_replace, 20)
        sel_df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
        out_data=pd.concat([out_data,sel_df],ignore_index=True,axis=1)
        sel_df= sel_df[~sel_df.ReviewText.str.contains("cid")]
        sel_df= sel_df[~sel_df.ReviewText.str.contains("et al")]
        out_data_clean=pd.concat([out_data_clean,sel_df],ignore_index=True,axis=1) #output file to csv
        
        fig, ax = plt.subplots(figsize=(15,7))
        sel_df.groupby('ReviewText').sum()['count'].sort_values(ascending=True).plot.barh(ax=ax,title='Top 20 keywords')
        
        fileName=file.split(".")[0]

        resFilename =  resDir + fileName + "_bigram" + ".png"
        
        matplotlib.use('Agg')

        fig.savefig(resFilename)
        
        out_data_clean.to_csv(consolidated, sep='\t', encoding='utf-8')
        
        plt.close(fig)
        
        print(resFilename)
        
    

C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/CN102666017B.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/CN102666017B_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/t.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/t_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US10137556.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US10137556_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US4770671.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US4770671_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US4964883.txt


...

C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US8142531_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US8764863.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US8764863_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US9200187.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US9200187_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US9566689.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US9566689_bigram.png
C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/datat/US9790410.txt


C:/Users/v9022828/Documents/Projects/Learn/TextAnalytics/Text/results/US9790410_bigram.png


In [ ]:
# Writing ouput file to CSV format

In [30]:
out_data_clean.to_csv('consolidated.csv')


In [16]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""
df_c.head()
#print(df_c.head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,United States Patent (19),"See discussions, stats, and author profiles fo...",( 12 ) United States Patent,United States Patent (19),United States Patent (19),United States Patent (19),United States Patent (19),United States Patent (19),s,United States Patent 19,United States Patent (19),United States Patent 19,United States Patent (19),United States Patent (19),United States Patent (19),(12),USOO8764863B2,(12) United States Patent,(12) United States Patent,THAT MOLT LLULITE
1,Monroe et al.,Deep learning and its applications to machine ...,Erickson,Monroe et al.,Morris et al.,Bloecher,Wood,Berg et al.,United States Patent (19),Lane et al.,Broberg et al.,Holmes et al.,Sheldon et al.,Garg et al.,Wood,United States Patent,(12) United States Patent,Bauer et al.,Iyengar,US009790410B2
2,(54),ArticleÂ Â inÂ Â Mechanical Systems and Signal...,( 54 ) SHAPED ABRASIVE PARTICLES WITH,(54),11) Patent Number:,(11) Patent Number:,(54) IMPREGNATION METHOD FOR,|I||||||||||I||,Wood et al.,(54),(54),||||||III,54 SOL-GELALUMNAABRASIVE WHEEL,54 FIRING SOL-GELALUMINA PARTICLES,54 METHOD FOR MAKING ABRASIVE GRAIN,Adefris et al.,Yener et al.,(10) Patent No.:,(10) Patent No.:,( 12 ) United States Patent
3,(75),DOI: 10.1016/j.ymssp.2018.05.050,LOW ROUNDNESS FACTOR,(75),(45) Date of Patent:,(45) Date of Patent:,TRANSFORMATION OF TRANSITION,"5,201,916",(54 HIGH DENSITY SOLGEL,(75),COATED ABRASIVE ARTICLE HAVING,USOO5549962A,WITH IMPROVED CORNER HOLDING,"(75) Inventors: Ajay K. Garg, Arup K. Khaund...",AND ABRASWE ARTICLES,(10) Patent No.:,(10) Patent No.:,(45) Date of Patent:,(45) Date of Patent:,Boden et al .
4,(73),CITATIONS,"( 75 ) Inventor : Dwight D . Erickson , Oakd...",(73),"4,964,883","5,039,311",ALUMINATO AALPHAALUMINA,"Apr. 13, 1993",ALUMNA-BASED ABRASIVE VTREOUS,GLASS-LIKE POLYSACCHARDE,DLUENT PARTICLES AND SHAPED,11,a-,"of Northborough; Lawrence E. Orne,","(75) Inventor: William P. Wood. Golden Valley,",(45) Date of Patent:,(45) Date of Patent:,"US 9.200,187 B2","US 9,566,689 B2",( 10 ) Patent No . :


## Plot word cloud

In [13]:
#data=df_4
for file in os.listdir(txtDir):
    fileName=file.split(".")[0]
    textFilename = txtDir + file
    resFilename =  resDir + fileName + ".png"
    text = open(textFilename,encoding='utf8').read()
    stop_words = ["cid","et al."] + list(STOPWORDS)
    wc =WordCloud(stopwords=stop_words)
    wordcloud = wc.generate(text)
    wordcloud.to_file(resFilename)


## End of presentation

In [ ]:
data = pd.read_csv(textFilename,header = None,delim_whitespace=True)

In [ ]:
fo = open(textFilename, "wb")


In [ ]:
fo.close()


# Top unigrams after removing stop words


In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df_s3, 20)
for word, freq in common_words:
    print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])

In [ ]:
submission = pd.DataFrame({"Keywords":df2['ReviewText'], "Count":df2['count']})
submission.to_csv("keywords.csv", index=False)

In [ ]:
df2.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review after removing stop words')

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words='english', ngram_range=(1,2))
tvec_weights = tvec.fit_transform(df_s3.stemmed.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

# Top trigrams after removing stop words


In [ ]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['text'], 20)
for word, freq in common_words:
    print(word, freq)
df6 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])

In [ ]:
df6.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in review after removing stop words')

# Top 20 part-of-speech tagging of review corpus


In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from textblob import TextBlob

In [ ]:
blob = TextBlob(str(df['text']))
pos_df = pd.DataFrame(blob.tags, columns = ['word' , 'pos'])
pos_df = pos_df.pos.value_counts()[:20]
pos_df.iplot(
    kind='bar',
    xTitle='POS',
    yTitle='count', 
    title='Top 20 Part-of-speech tagging for review corpus')

# Topic Modeling with LSA (Latent Semantic Analysis)
#https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05

In [ ]:
reindexed_data = df_s
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reindexed_data = reindexed_data.values
document_term_matrix = tfidf_vectorizer.fit_transform(reindexed_data)

In [ ]:
n_topics = 6
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)

In [ ]:
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

In [ ]:
lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

In [ ]:
def get_top_n_words(n, keys, document_term_matrix, tfidf_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = tfidf_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [ ]:
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:

top_3_words = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
labels = ['Topic {}: \n'.format(i) + top_3_words[i] for i in lsa_categories]

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lsa_categories, lsa_counts);
ax.set_xticks(lsa_categories);
ax.set_xticklabels(labels);
ax.set_ylabel('Number of review text');
ax.set_title('LSA topic counts');
plt.show();

# Text Summarization

In [ ]:
from gensim.summarization.summarizer import summarize


In [ ]:
extracted_text

In [ ]:
print(summarize(extracted_text))

## Word cloud

In [ ]:
dff = pd.DataFrame(dict(A=[5,3,5,6], C=["foo","bar","fooXYZbar", "bat"]))

In [ ]:
dff.head()

In [ ]:
dff[~dff.C.str.contains("XYZ")]

In [ ]:

stopwords= set(STOPWORDS)
new_stopwords=stopwords.union("cid")


In [ ]:


wc = WordCloud(background_color="white", 
               max_words=2000, 
               mask=mask,
               min_font_size =12, 
               max_font_size=20, 
               relative_scaling = 0.5, 
               stopwords=new_stopwords,
               normalize_plurals= True)

In [ ]:
for file in os.listdir(txtDir):
    fileName=file.split(".")[0]
    textFilename = txtDir + file
    resFilename =  resDir + fileName + ".png"
    text = open(textFilename,encoding='utf8').read()
    stop_words = ["cid","et al"] + list(STOPWORDS)
    wc =WordCloud(stopwords=stop_words)
    wordcloud = wc.generate(text)
    wordcloud.to_file(resFilename)


In [ ]:
new_stopwords